Section header classiifcation using keras and tensorflow
https://github.com/fchollet/keras/tree/master/examples
https://github.com/bhaveshoswal/CNN-text-classification-keras/blob/master/model.py

https://github.com/offbit/char-models/blob/master/doc-rnn2.py

In [ ]:
#from __future__ import print_function
from keras.layers import Input, Dense, Embedding, merge, Convolution2D, MaxPooling2D, Dropout, Activation
from sklearn.cross_validation import train_test_split
from keras.layers.core import Reshape, Flatten
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model,Sequential
from collections import defaultdict
import csv
import numpy as np
import skflow
from keras.preprocessing import sequence
from keras.layers import Embedding, LSTM,Lambda,Conv1D,MaxPooling1D,Bidirectional,TimeDistributed
from keras.layers import Conv1D, GlobalMaxPooling1D
import sys
csv.field_size_limit(sys.maxsize)
from keras.datasets import imdb
import tensorflow as tf
import keras


In [ ]:
#datafile= "data_for_weka_all.csv"
datafile1= "../s3/training_data/train_sample/train.part1.csv"
datafile2= "../s3/training_data/train/train.part2.csv"
datafile3= "../s3/training_data/test/test.part501.csv"

duplicate_samples=1
duplicate_samples_pos=1

checkpoint = None

In [ ]:
# convert section header dataset for rnn neural network

# data for training and test where we may duplicate positive samples. We may also split the data 
# into two set. training and test. This ae for moel tesing. 
sh_dataset = defaultdict(lambda : None)
sh_dataset['target_names'] =['no','yes']
sh_dataset['target'] =[]
sh_dataset['data'] =[]

# real  data is for tesing the model on a different sample set
real_dataset = defaultdict(lambda : None)
real_dataset['target_names'] =['no','yes']
real_dataset['target'] =[]
real_dataset['data'] =[]



with open(datafile1, 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        for i in range(duplicate_samples):
            if float(row['class']) > 0:
            #if row['class'] =="yes":
                for j in range(duplicate_samples_pos):
                    sh_dataset['target'].append(1)
                    sh_dataset['data'].append(row['text'])
            else:
                sh_dataset['target'].append(0)  
                sh_dataset['data'].append(row['text'])
with open(datafile2, 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        for i in range(duplicate_samples):
            if float(row['class']) > 0:
            #if row['class'] =="yes":
                for j in range(duplicate_samples_pos):
                    sh_dataset['target'].append(1)
                    sh_dataset['data'].append(row['text'])
            else:
                sh_dataset['target'].append(0)  
                sh_dataset['data'].append(row['text'])

with open(datafile3, 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        for i in range(duplicate_samples):
            if float(row['class']) > 0:
            #if row['class'] =="yes":
                for j in range(1): # as in relad test, we don't need to dupliate positive samples
                    real_dataset['target'].append(1)
                    real_dataset['data'].append(row['text'])
            else:
                real_dataset['target'].append(0)  
                real_dataset['data'].append(row['text'])
            
        

In [ ]:
#X_train, y_train = sh_dataset['data'], sh_dataset['target']
#X_test,  y_test = real_dataset['data'], real_dataset['target']

In [ ]:
MAX_DOCUMENT_LENGTH = 100

char_processor = skflow.preprocessing.ByteProcessor(MAX_DOCUMENT_LENGTH)
X_train = np.array(list(char_processor.fit_transform(X_train)))
X_test = np.array(list(char_processor.transform(X_test)))

print X_train.shape
#print y_train.shape
print X_test.shape
#print y_test.shape

In [ ]:
txt = ''
docs = []
sentences = []
sentiments = []

for cont, sentiment in zip(sh_dataset['data'],sh_dataset['target']):
    sentences = cont
    docs.append(sentences)
    sentiments.append(sentiment)

for doc in docs:
    for s in doc:
        txt += s
chars = set(txt)
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))    

In [ ]:
maxlen = 16
max_sentences = 5

X = np.ones((len(docs), max_sentences, maxlen), dtype=np.int64) * -1
y = np.array(sentiments)

for i, doc in enumerate(docs):
    for j, sentence in enumerate(doc):
        if j < max_sentences:
            for t, char in enumerate(sentence[-maxlen:]):
                X[i, j, (maxlen-1-t)] = char_indices[char]

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accuracies = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracies.append(logs.get('acc'))



In [ ]:
def binarize(x, sz=71):
    return tf.to_float(tf.one_hot(x, sz, on_value=1, off_value=0, axis=-1))
def binarize_outshape(in_shape):
    return in_shape[0], in_shape[1], 71

In [ ]:
X_train = X[:10000]
X_test = X[10000:]

y_train = y[:10000]
y_test = y[10000:]

filter_length = [5, 3, 3]
nb_filter = [196, 196, 256]
pool_length = 1


in_sentence = Input(shape=(maxlen,), dtype='int64')
# binarize function creates a onehot encoding of each character index
embedded = Lambda(binarize, output_shape=binarize_outshape)(in_sentence)


In [ ]:
for i in range(len(nb_filter)):
    embedded = Conv1D(nb_filter=nb_filter[i],
                            filter_length=filter_length[i],
                            border_mode='valid',
                            activation='relu',
                            init='glorot_normal',
                            subsample_length=1)(embedded)

    embedded = Dropout(0.1)(embedded)
    embedded = MaxPooling1D(pool_length=pool_length)(embedded)

In [ ]:
bi_lstm_sent = Bidirectional(LSTM(128, return_sequences=False, dropout=0.15, recurrent_dropout=0.15, implementation=0))(embedded)

# sent_encode = merge([forward_sent, backward_sent], mode='concat', concat_axis=-1)
sent_encode = Dropout(0.3)(bi_lstm_sent)
# sentence encoder
encoder = Model(inputs=in_sentence, outputs=sent_encode)
encoder.summary()

In [ ]:
document = Input(shape=(max_sentences, maxlen), dtype='int64')
encoded = TimeDistributed(encoder)(document)
# encoded: sentences to bi-lstm for document encoding 
b_lstm_doc = \
    Bidirectional(LSTM(128, return_sequences=False, dropout=0.15, recurrent_dropout=0.15, implementation=0))(encoded)

output = Dropout(0.3)(b_lstm_doc)
output = Dense(128, activation='relu')(output)
output = Dropout(0.3)(output)
output = Dense(1, activation='sigmoid')(output)

model = Model(inputs=document, outputs=output)

model.summary()

In [ ]:
if checkpoint:
    model.load_weights(checkpoint)

check_cb = keras.callbacks.ModelCheckpoint('checkpoints/' + "checkpoints" + '.{epoch:02d}-{val_loss:.2f}.hdf5',
                                           monitor='val_loss',verbose=0, save_best_only=True, mode='min')

earlystop_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='auto')
#history = LossHistory()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=5,
          epochs=5, shuffle=True, callbacks=[earlystop_cb, check_cb, history])

# just showing access to the history object
#print history.losses
#print history.accuracies
